In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
subject = 'Paraná - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [8]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
droped_list = ['Paraná - Produção de Cimento (t)',
               'Paraná - IDH',
               'Carteira de Crédito - FGTS (R$)',
               'Taxa Selic (%)',
               'Carteira de Crédito – Home equity (R$)',
               'Quantidade de imóves – Isolado',
               'Paraná - IDH Renda',
               'Atmospheric Pressure(mB)',
               'Número de Apartamentos',
               'Inadimplência - SFH (%)',
               'Quantidade de imóves - Condomínio']
data = data.drop(droped_list, axis=1)
data

,Ano,Paraná - Desemprego,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Precipitation (mm/day),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Consumo de Cimento (t)
0,2008,8.033948,2.148670e+08,1.122835e+07,17.780948,1.878837e+08,0.745648,0.834122,5.483335e+08,2.546644e+09,...,0.415443,-7723.574973,2.868108,17.558665,1226.296848,18.012164,13.653739,78.027482,2.236583,3001.521000
1,2009,8.063370,2.163260e+08,1.133139e+07,17.787925,1.879466e+08,0.748601,0.835310,5.302764e+08,2.679945e+09,...,0.446751,-8961.781709,2.305885,16.652522,1217.653383,17.954863,13.668549,78.238643,2.155189,3092.068000
2,2010,8.026843,2.218671e+08,1.138377e+07,17.886000,1.893914e+08,0.751387,0.835940,5.046425e+08,2.891930e+09,...,0.464446,-9895.682385,1.958246,17.928528,1194.060192,17.703414,13.001948,76.311573,2.092804,3749.448000
3,2011,8.013806,2.270561e+08,1.138563e+07,17.967281,1.906364e+08,0.753998,0.836020,4.854622e+08,3.091988e+09,...,0.468365,-9989.170714,2.079088,18.661309,1182.416452,17.710041,12.630186,74.615878,2.043995,4174.942000
4,2012,6.472496,2.317175e+08,1.133915e+07,18.011636,1.916091e+08,0.755977,0.835445,4.867661e+08,3.189506e+09,...,0.489965,-9429.326328,2.183282,19.310061,1176.491125,17.874291,12.612094,73.865245,1.941425,4522.969000
5,2013,7.011799,2.358269e+08,1.122742e+07,18.040420,1.923917e+08,0.759142,0.834565,5.225848e+08,3.093872e+09,...,0.553888,-7157.441850,2.403445,19.233593,1170.878793,18.105264,12.720105,73.487865,1.844730,4578.716000
6,2014,7.462443,2.386532e+08,1.115283e+07,18.028052,1.924772e+08,0.760987,0.833112,5.676413e+08,2.779340e+09,...,0.630875,-3453.461327,2.748397,20.725057,1150.562966,18.275455,13.157699,74.605015,1.851460,4341.821857
7,2015,7.813261,2.410111e+08,1.107686e+07,18.031270,1.926648e+08,0.762152,0.831122,5.179392e+08,2.561731e+09,...,0.554104,-1442.919387,2.501529,22.054778,1145.712487,18.177724,13.346348,75.170150,1.905966,4108.786286
8,2016,7.840669,2.424341e+08,1.101317e+07,18.026717,1.926914e+08,0.771114,0.838386,4.524144e+08,2.516859e+09,...,0.422483,-3128.377228,2.073464,22.315526,1158.573328,18.115048,13.261122,74.043778,1.933375,3869.334857
9,2017,7.553100,2.428100e+08,1.097443e+07,18.022550,1.926406e+08,0.770943,0.835784,4.213689e+08,2.497710e+09,...,0.369819,-5386.337108,2.177386,21.977949,1160.957791,18.281050,13.470416,73.546507,1.889856,3764.518000


In [9]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Paraná - Desemprego,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Valor Imóveis - Compra (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Precipitation (mm/day),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.033948,2.148670e+08,1.122835e+07,17.780948,1.878837e+08,0.745648,0.834122,5.483335e+08,2.546644e+09,1.874883e+10,...,5.850439e+05,0.415443,-7723.574973,2.868108,17.558665,1226.296848,18.012164,13.653739,78.027482,2.236583
1,8.063370,2.163260e+08,1.133139e+07,17.787925,1.879466e+08,0.748601,0.835310,5.302764e+08,2.679945e+09,2.055015e+10,...,6.367470e+05,0.446751,-8961.781709,2.305885,16.652522,1217.653383,17.954863,13.668549,78.238643,2.155189
2,8.026843,2.218671e+08,1.138377e+07,17.886000,1.893914e+08,0.751387,0.835940,5.046425e+08,2.891930e+09,2.342247e+10,...,7.413001e+05,0.464446,-9895.682385,1.958246,17.928528,1194.060192,17.703414,13.001948,76.311573,2.092804
3,8.013806,2.270561e+08,1.138563e+07,17.967281,1.906364e+08,0.753998,0.836020,4.854622e+08,3.091988e+09,2.604265e+10,...,8.874467e+05,0.468365,-9989.170714,2.079088,18.661309,1182.416452,17.710041,12.630186,74.615878,2.043995
4,6.472496,2.317175e+08,1.133915e+07,18.011636,1.916091e+08,0.755977,0.835445,4.867661e+08,3.189506e+09,2.708753e+10,...,1.063930e+06,0.489965,-9429.326328,2.183282,19.310061,1176.491125,17.874291,12.612094,73.865245,1.941425
5,7.011799,2.358269e+08,1.122742e+07,18.040420,1.923917e+08,0.759142,0.834565,5.225848e+08,3.093872e+09,2.523394e+10,...,1.259494e+06,0.553888,-7157.441850,2.403445,19.233593,1170.878793,18.105264,12.720105,73.487865,1.844730
6,7.462443,2.386532e+08,1.115283e+07,18.028052,1.924772e+08,0.760987,0.833112,5.676413e+08,2.779340e+09,1.954361e+10,...,1.462882e+06,0.630875,-3453.461327,2.748397,20.725057,1150.562966,18.275455,13.157699,74.605015,1.851460
7,7.813261,2.410111e+08,1.107686e+07,18.031270,1.926648e+08,0.762152,0.831122,5.179392e+08,2.561731e+09,1.327596e+10,...,1.662837e+06,0.554104,-1442.919387,2.501529,22.054778,1145.712487,18.177724,13.346348,75.170150,1.905966
8,7.840669,2.424341e+08,1.101317e+07,18.026717,1.926914e+08,0.771114,0.838386,4.524144e+08,2.516859e+09,1.011370e+10,...,1.848104e+06,0.422483,-3128.377228,2.073464,22.315526,1158.573328,18.115048,13.261122,74.043778,1.933375
9,7.553100,2.428100e+08,1.097443e+07,18.022550,1.926406e+08,0.770943,0.835784,4.213689e+08,2.497710e+09,8.456309e+09,...,2.007424e+06,0.369819,-5386.337108,2.177386,21.977949,1160.957791,18.281050,13.470416,73.546507,1.889856


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     3092.068000
1     3749.448000
2     4174.942000
3     4522.969000
4     4578.716000
5     4341.821857
6     4108.786286
7     3869.334857
8     3764.518000
9     3773.354000
10    3786.974000
11    4235.532000
12    4784.413000
13    4538.256000
14            NaN
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [11]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input 

,Paraná - Desemprego,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Valor Imóveis - Compra (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Precipitation (mm/day),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.954430,-1.806944,0.348365,-2.042958,-1.906928,-1.621982,-0.042462,1.204752,-0.990234,0.166091,...,-1.377247,-0.578425,-0.569731,2.026352,-1.266771,1.940409,-0.268956,1.137644,1.835993,2.079287
1,1.017927,-1.661370,1.009092,-1.965533,-1.871184,-1.293295,0.429908,0.820657,-0.546688,0.428911,...,-1.287537,-0.194147,-0.838980,0.085558,-1.745665,1.579454,-0.539264,1.176616,1.956737,1.461930
2,0.939098,-1.108495,1.345008,-0.877208,-1.049479,-0.983091,0.680031,0.275392,0.158672,0.847995,...,-1.106127,0.023036,-1.042057,-1.114490,-1.071300,0.594192,-1.725446,-0.577589,0.854816,0.988761
3,0.910963,-0.590757,1.356922,0.024771,-0.341452,-0.692424,0.712066,-0.132596,0.824346,1.230291,...,-0.852548,0.071145,-1.062386,-0.697344,-0.684027,0.107944,-1.694185,-1.555909,-0.114802,0.618558
4,-2.415320,-0.125660,1.058841,0.516968,0.211766,-0.472052,0.483303,-0.104860,1.148829,1.382743,...,-0.546332,0.336259,-0.940648,-0.337667,-0.341164,-0.139500,-0.919352,-1.603519,-0.544022,-0.159410
5,-1.251457,0.284361,0.342383,0.836388,0.656850,-0.119710,0.133711,0.657046,0.830615,1.112297,...,-0.207010,1.120857,-0.446625,0.422337,-0.381577,-0.373874,0.170234,-1.319280,-0.759813,-0.892823
6,-0.278927,0.566357,-0.135907,0.699136,0.705450,0.085716,-0.443735,1.615453,-0.215960,0.282052,...,0.145888,2.065797,0.358807,1.613108,0.406656,-1.222272,0.973093,-0.167721,-0.121014,-0.841773
7,0.478169,0.801625,-0.623089,0.734849,0.812173,0.215483,-1.234902,0.558230,-0.940033,-0.632425,...,0.492830,1.123510,0.796000,0.760923,1.109410,-1.424830,0.512055,0.328724,0.202137,-0.428361
8,0.537319,0.943598,-1.031518,0.684323,0.827306,1.213246,1.652199,-0.835563,-1.089341,-1.093812,...,0.814285,-0.492014,0.429496,-0.716758,1.247214,-0.887756,0.216390,0.104443,-0.441935,-0.220470
9,-0.083280,0.981108,-1.279958,0.638087,0.798387,1.194161,0.618336,-1.495939,-1.153056,-1.335633,...,1.090722,-1.138419,-0.061498,-0.358019,1.068806,-0.788179,0.999483,0.655217,-0.726280,-0.550555


In [12]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     3092.068000
1     3749.448000
2     4174.942000
3     4522.969000
4     4578.716000
5     4341.821857
6     4108.786286
7     3869.334857
8     3764.518000
9     3773.354000
10    3786.974000
11    4235.532000
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [13]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Paraná - Desemprego,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Valor Imóveis - Compra (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Precipitation (mm/day),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.734266,0.465024,-0.914446,-0.223777,0.114326,1.035451,-2.637193,1.108405,2.635823,-0.189959,...,1.512767,1.049932,0.964765,-1.482779,0.964616,-0.540552,0.224215,0.839814,-2.237536,-1.155068


In [14]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    4784.413
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3869691706, 1263233749, 1379522781, 1091879811, 4236154409, 3107799860, 3386360306, 1841797525, 4232132094, 2487497787, 1698631269, 3574932457, 1606909042, 2369747352, 3744218731, 4162990560, 3968218151, 207670431, 3398225078, 4009115991, 910937534, 3624527153, 1740184258, 2705506929, 3925977733, 911838982, 1135347585, 2441590784, 1643163028, 3452680137, 1634011512, 3222281195, 61462565, 3246495738, 520721007, 2645370832, 4048007350, 1339284936, 1700012463, 2549148509, 3027278029, 2877398016, 1806233727, 882900864, 1996760662, 3739065246, 4131331043, 950477504, 4032281456, 2007777267, 2077717288, 4120908119, 2675178334, 149915212, 1707604676, 2564086078, 2805947696, 3325554961, 725344836, 322493350, 2529934440, 2554296574, 196876554, 2100740316, 2663775612, 402910537, 1976574464, 3309383173, 171001742, 2244935008, 3106495183, 3524365471, 516336999, 2855647898, 1131022052, 1932016310, 608083129, 3375701751, 1463919897, 1255951953, 3116566651, 2463577380, 1095141786, 582020884, 37702285

val_loss: 6479.62109375


Step: 90 ___________________________________________
val_loss: 45243.34765625


Step: 91 ___________________________________________
val_loss: 91436.6953125


Step: 92 ___________________________________________
val_loss: 100428.1796875


Step: 93 ___________________________________________
val_loss: 54554.40625


Step: 94 ___________________________________________
val_loss: 47027.56640625


Step: 95 ___________________________________________
val_loss: 78290.25


Step: 96 ___________________________________________
val_loss: 44359.1875


Step: 97 ___________________________________________
val_loss: 83976.140625


Step: 98 ___________________________________________
val_loss: 89716.1875


Step: 99 ___________________________________________
val_loss: 57873.25390625


final_seed: 3770228535


In [19]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 475ms/step - loss: 16155425.0000 - val_loss: 17522534.0000
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 15904543.0000 - val_loss: 19199634.0000
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 15730408.0000 - val_loss: 16541829.0000
Epoch 4/10000
1/1 [==============================] - 0s 12ms/step - loss: 14478049.0000 - val_loss: 17222256.0000
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 14869837.0000 - val_loss: 16991436.0000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 14585283.0000 - val_loss: 16182960.0000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 14366595.0000 - val_loss: 16524757.0000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 13793931.0000 - val_loss: 14707858.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 13850163.0000

1/1 [==============================] - 0s 13ms/step - loss: 3931620.0000 - val_loss: 5006254.5000
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 3845605.7500 - val_loss: 4889711.0000
Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 3782817.2500 - val_loss: 4844985.5000
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 3710494.7500 - val_loss: 4751384.5000
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 3635081.2500 - val_loss: 4626945.0000
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 3570334.2500 - val_loss: 4605794.0000
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 3502182.7500 - val_loss: 4511398.0000
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 3435481.2500 - val_loss: 4439217.5000
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 3276536.2500 - val_loss: 4504886.000

Epoch 218/10000
1/1 [==============================] - 0s 13ms/step - loss: 349253.4688 - val_loss: 618476.2500
Epoch 219/10000
1/1 [==============================] - 0s 14ms/step - loss: 354267.1250 - val_loss: 565378.8125
Epoch 220/10000
1/1 [==============================] - 0s 13ms/step - loss: 349721.1250 - val_loss: 571323.0000
Epoch 221/10000
1/1 [==============================] - 0s 13ms/step - loss: 345722.5312 - val_loss: 561137.0625
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 331248.8750 - val_loss: 575006.0000
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 338284.8750 - val_loss: 543712.5000
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 334400.9688 - val_loss: 544244.8125
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 330734.1562 - val_loss: 536096.0625
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 327151.1875 - val_loss: 5304

1/1 [==============================] - 0s 13ms/step - loss: 162072.9531 - val_loss: 174617.6719
Epoch 365/10000
1/1 [==============================] - 0s 13ms/step - loss: 161847.2969 - val_loss: 173737.7500
Epoch 366/10000
1/1 [==============================] - 0s 13ms/step - loss: 161626.1094 - val_loss: 172868.7656
Epoch 367/10000
1/1 [==============================] - 0s 13ms/step - loss: 158191.2969 - val_loss: 179251.5625
Epoch 368/10000
1/1 [==============================] - 0s 13ms/step - loss: 161283.5781 - val_loss: 169234.6094
Epoch 369/10000
1/1 [==============================] - 0s 13ms/step - loss: 161007.2969 - val_loss: 170933.3906
Epoch 370/10000
1/1 [==============================] - 0s 13ms/step - loss: 160797.6094 - val_loss: 169395.9531
Epoch 371/10000
1/1 [==============================] - 0s 14ms/step - loss: 160596.9219 - val_loss: 168777.3438
Epoch 372/10000
1/1 [==============================] - 0s 13ms/step - loss: 160400.6094 - val_loss: 167915.0781
Epoch 37

1/1 [==============================] - 0s 18ms/step - loss: 151409.6406 - val_loss: 114378.7109
Epoch 511/10000
1/1 [==============================] - 0s 18ms/step - loss: 151396.1875 - val_loss: 114204.5625
Epoch 512/10000
1/1 [==============================] - 0s 19ms/step - loss: 151382.9844 - val_loss: 114032.3516
Epoch 513/10000
1/1 [==============================] - 0s 19ms/step - loss: 151370.0469 - val_loss: 113861.9219
Epoch 514/10000
1/1 [==============================] - 0s 23ms/step - loss: 151357.3594 - val_loss: 113693.4375
Epoch 515/10000
1/1 [==============================] - 0s 18ms/step - loss: 151344.9531 - val_loss: 113526.7109
Epoch 516/10000
1/1 [==============================] - 0s 18ms/step - loss: 151332.7656 - val_loss: 113361.7578
Epoch 517/10000
1/1 [==============================] - 0s 19ms/step - loss: 151320.8281 - val_loss: 113198.5703
Epoch 518/10000
1/1 [==============================] - 0s 19ms/step - loss: 151309.1406 - val_loss: 113037.1406
Epoch 51

...

Epoch 951/10000
1/1 [==============================] - 0s 14ms/step - loss: 151628.4219 - val_loss: 93055.6797
Epoch 952/10000
1/1 [==============================] - 0s 13ms/step - loss: 150804.1719 - val_loss: 99703.4609
Epoch 953/10000
1/1 [==============================] - 0s 13ms/step - loss: 150739.5469 - val_loss: 97811.1875
Epoch 954/10000
1/1 [==============================] - 0s 13ms/step - loss: 150734.4375 - val_loss: 98331.5469
Epoch 955/10000
1/1 [==============================] - 0s 13ms/step - loss: 150734.0156 - val_loss: 98178.0312
Epoch 956/10000
1/1 [==============================] - 0s 14ms/step - loss: 150733.9844 - val_loss: 98213.5312
Epoch 957/10000
1/1 [==============================] - 0s 14ms/step - loss: 150733.9531 - val_loss: 98196.0859
Epoch 958/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.9375 - val_loss: 98193.4844
Epoch 959/10000
1/1 [==============================] - 0s 13ms/step - loss: 181315.0000 - val_loss: 117056.3672


1/1 [==============================] - 0s 14ms/step - loss: 150733.1406 - val_loss: 97747.0625
Epoch 1098/10000
1/1 [==============================] - 0s 14ms/step - loss: 150733.1406 - val_loss: 97745.8359
Epoch 1099/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1406 - val_loss: 97744.6172
Epoch 1100/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1406 - val_loss: 97743.2422
Epoch 1101/10000
1/1 [==============================] - 0s 14ms/step - loss: 150733.1406 - val_loss: 97742.3281
Epoch 1102/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1406 - val_loss: 97740.8047
Epoch 1103/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1406 - val_loss: 97739.7344
Epoch 1104/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1250 - val_loss: 97738.5156
Epoch 1105/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1406 - val_loss: 97737.4453
Epoch 110

1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97678.0703
Epoch 1244/10000
1/1 [==============================] - 0s 14ms/step - loss: 150733.1094 - val_loss: 97677.4609
Epoch 1245/10000
1/1 [==============================] - 0s 14ms/step - loss: 150733.1094 - val_loss: 97677.0000
Epoch 1246/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97676.5469
Epoch 1247/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97676.0859
Epoch 1248/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97675.6328
Epoch 1249/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97675.1719
Epoch 1250/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97674.7109
Epoch 1251/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97674.2578
Epoch 125

1/1 [==============================] - 0s 14ms/step - loss: 150733.1094 - val_loss: 97662.2031
Epoch 1390/10000
1/1 [==============================] - 0s 14ms/step - loss: 150733.1094 - val_loss: 97661.5938
Epoch 1391/10000
1/1 [==============================] - 0s 14ms/step - loss: 150733.1094 - val_loss: 97661.4375
Epoch 1392/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97661.1328
Epoch 1393/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97660.8281
Epoch 1394/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97660.5234
Epoch 1395/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.0938 - val_loss: 97660.2188
Epoch 1396/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97659.9141
Epoch 1397/10000
1/1 [==============================] - 0s 13ms/step - loss: 150733.1094 - val_loss: 97659.6094
Epoch 139

Epoch 1461: early stopping


In [20]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[4784.413](test_target) - [[4204.2905]](prediction) = 580.1224726562496


In [21]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [22]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Paraná - Desemprego,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Valor Imóveis - Compra (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Precipitation (mm/day),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.470460,-0.935098,-1.334379,-0.778595,-0.751752,-1.236383,-1.329522,1.148205,-1.122043,-1.121418,...,-1.066520,-1.321274,1.277766,1.308949,0.333480,1.000190,0.909918,0.683149,0.581225,1.275014
1,1.390219,-0.451251,0.261510,-0.633122,-0.661502,0.023618,0.247295,0.140893,-0.184474,-0.185492,...,-0.271041,0.223972,-0.114016,-0.190804,-1.356947,0.365766,0.482610,0.730797,0.825914,-0.107644
2,-0.919759,1.386349,1.072868,1.411717,1.413253,1.212765,1.082228,-1.289098,1.306518,1.306910,...,1.337562,1.097302,-1.163750,-1.118144,1.023467,-1.365956,-1.392529,-1.413946,-1.407140,-1.167370


train_target:


,Paraná - Consumo de Cimento (t)
0,3092.068
1,3749.448
2,4174.942


test_input:


...

,Paraná - Desemprego,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Valor Imóveis - Compra (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Precipitation (mm/day),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.734266,0.465024,-0.914446,-0.223777,0.114326,1.035451,-2.637193,1.108405,2.635823,-0.189959,...,1.512767,1.049932,0.964765,-1.482779,0.964616,-0.540552,0.224215,0.839814,-2.237536,-1.155068


test_target:


,Paraná - Consumo de Cimento (t)
12,4784.413


1/1 [==============================] - 0s 34ms/step
Error: 580.1224726562496




[4175.03466796875,
 4348.943359375,
 4438.3583984375,
 3880.53857421875,
 4156.66796875,
 3869.377685546875,
 3764.78076171875,
 3802.462890625,
 3786.931640625,
 4204.29052734375]

In [23]:
display(targets)
display(predictions)

[4522.969,
 4578.716,
 4341.821857142857,
 4108.786285714286,
 3869.334857142857,
 3764.518,
 3773.354,
 3786.974,
 4235.532,
 4784.413]

[4175.03466796875,
 4348.943359375,
 4438.3583984375,
 3880.53857421875,
 4156.66796875,
 3869.377685546875,
 3764.78076171875,
 3802.462890625,
 3786.931640625,
 4204.29052734375]

In [24]:
mae = mean_absolute_error(predictions, targets)
mae

234.74689835379473

In [25]:
porcentage = mae/np.mean(targets)
porcentage

0.056204698409455384